In [ ]:
from azure.storage.blob import BlobServiceClient

# === CONFIGURATION ===
connection_string = "xyz"  # 🔐 Replace this with your actual Azure Storage connection string
container_name = "raw-data"
local_file_path = r"A:\DE\NYC-Taxi-ETL-Azure\taxi_zone_lookup.csv"
blob_name = "taxi_zone_lookup.csv"  # How the file will appear in Azure

# === UPLOAD TO BLOB STORAGE ===
try:
    blob_service_client = BlobServiceClient.from_connection_string(connection_string)
    blob_client = blob_service_client.get_blob_client(container=container_name, blob=blob_name)

    with open(local_file_path, "rb") as data:
        blob_client.upload_blob(data, overwrite=True)

    print(f"✅ Successfully uploaded '{blob_name}' to Azure container '{container_name}'.")

except Exception as e:
    print("❌ Upload failed:", e)


✅ Successfully uploaded 'taxi_zone_lookup.csv' to Azure container 'raw-data'.


In [3]:
import pandas as pd
from azure.storage.blob import BlobServiceClient
from io import BytesIO

# === CONFIG ===
connection_string = "DefaultEndpointsProtocol=https;AccountName=ambinyctaxistorage;AccountKey=ym+nDxr6ssJOZ/1x/YZB6m1OIINYPw45tayGFDnTjXnlia/sUe9alZp4X78/Aksbfp1f+6HWQde2+AStH4p5RQ==;EndpointSuffix=core.windows.net"
raw_container = "raw-data"
transformed_container = "transformed-data"
blob_name = "taxi_zone_lookup.csv"

# === Connect to Blob Service ===
blob_service_client = BlobServiceClient.from_connection_string(connection_string)

# === Step 1: Read Raw CSV from Azure Blob ===
blob_client = blob_service_client.get_blob_client(container=raw_container, blob=blob_name)
download_stream = blob_client.download_blob()
raw_data = download_stream.readall()

# Read into Pandas
df = pd.read_csv(BytesIO(raw_data))

# === Step 2: Clean / Transform ===
# Example transformations
df.columns = df.columns.str.lower()  # Lowercase column names
df = df.dropna()  # Drop rows with missing values

# === Step 3: Save Transformed CSV in Memory ===
output = BytesIO()
df.to_csv(output, index=False)
output.seek(0)

# === Step 4: Upload to Transformed Container ===
# Make sure the container exists (create if not)
try:
    blob_service_client.create_container(transformed_container)
except:
    pass  # Container probably already exists

transformed_blob_name = "taxi_zone_lookup_transformed.csv"
transformed_blob_client = blob_service_client.get_blob_client(container=transformed_container, blob=transformed_blob_name)
transformed_blob_client.upload_blob(output, overwrite=True)

print(f"✅ Transformed file uploaded as '{transformed_blob_name}' in container '{transformed_container}'")


✅ Transformed file uploaded as 'taxi_zone_lookup_transformed.csv' in container 'transformed-data'
